# Defining classes

In [1]:
class MyClass:
    # An instance method
    def set(self,value):
        # A variable defined in each instance
        self.val = value
    def get(self):
        return self.val

In [2]:
a = MyClass() # An instance of MyClass
a.set(2222)
a.get()

2222

Class variables are shared across all instances of the class

In [3]:
class MyClassCounter:
    readCounter = 0 # A class variable
    writeCounter = 0 # A class variable
    def set(self,value):
        # A class variable for *all* instances.
        MyClassCounter.writeCounter += 1
        self.val = value
    def get(self):
        # A class variable for *all* instances.
        MyClassCounter.readCounter += 1
        return self.val

In [4]:
storeA = MyClassCounter()
storeA.set(32)
print(storeA.get()) # 32
storeB = MyClassCounter()
storeB.set(16)
print(MyClassCounter.readCounter) # 1
print(MyClassCounter.writeCounter) # 2

32
1
2


We can overload operators and keywords for instances of a class, for example by defining \_\_add\_\_ to implement addition.

In [5]:
class MyClassAdd:
    def __init__(self,value):
        self.val = value
        # A class method
    def set(self,value):
        # A variable defined in each instance
        self.val = value
    def get(self):
        return self.val
    def __add__(self,b):
        return MyClassAdd(self.val+b.get())

In [6]:
a = MyClassAdd(2)
b = MyClassAdd(3)
c=a+b
c.get()

5

# An example class - sets of unordered pairs

We can build a class that treats pairs $(a,b)$ and $(b,a)$ as equivalent, by representing pairs as $(c,d)$ with $c = min(a,b)$ and $d = max(a,b)$. This gives a canonical representation that is the same regardless of the ordering of $a$ and $b$.

In [7]:
class upairset:
    def __init__(self):
        self.pairs = set()
    def add(self,pair):
        a,b = pair
        if a<b:
            self.pairs.add((a,b))
        else:
            self.pairs.add((b,a))      
    def __contains__(self,pair):
        a,b = pair
        if a<b:
            return (a,b) in self.pairs
        else:
            return (b,a) in self.pairs

In [8]:
edges = upairset()
edges.add((5,2))
(2,5) in edges

True

# A simple network class

Storing edges in a Python list, and with a method to parse a network from a tsv file.

In [9]:
class Network:
    def __init__(self):
        self.edges = []
    def readFile(self,filename):
        with open(filename,'r') as f:
            for l in f:
                a,b = l.split("\t")
                e = (int(a),int(b))
                self.addEdge(e)
    def addEdge(self,e):
        self.edges.append(e)

We can make a better Network class using a set of unordered pairs to store the edge list. We can do this by inheriting the readFile method from the network class above, and overwriting the \_\_init\_\_ and addEdge methods.

We can also define a \_\_contains\_\_ method for the class, that simply uses the method of the upairset class.

In [10]:
class NetworkEdgeList(Network):
    def __init__(self):
        self.edges = upairset()
    def addEdge(self,e):
        self.edges.add(e) 
    def __contains__(self,e):
        return e in self.edges

We can do something similar for a neighbour list class, using sets to represent neighbour lists.

In [11]:
class NetworkNeighbourList(Network):
    def __init__(self):
        self.neighbours = {}
    def addEdge(self,e):
        a,b = e
        if a in self.neighbours:
            self.neighbours[a].add(b)
        else:
            self.neighbours[a] = {b}
        if b in self.neighbours:
            self.neighbours[b].add(a)
        else:
            self.neighbours[b] = {a}        
    def __contains__(self,e):
        a,b = e
        return b in self.neighbours[a]
    def neighbourList(self,a):
        return self.neighbours[a]

Testing out our various Network classes:

In [12]:
n = Network()
n.readFile("dolphins.tsv")
n.edges[:4] # First four edges

[(9, 4), (10, 6), (10, 7), (11, 1)]

In [13]:
ne = NetworkEdgeList()
ne.readFile("dolphins.tsv")
(4,9) in ne.edges

True

In [14]:
nn = NetworkNeighbourList()
nn.readFile("dolphins.tsv")
nn.neighbourList(10)

{6, 7, 14, 18, 33, 42, 58}

# Drawing the network with NetworkX and Bokeh

We can use the Bokeh visualisation library to draw our network, but we need to use the NetworkX library to produce a layout (set of node coordinates) for the network.

We can extend our simple network class with a method to convert the network to a networkx object.

In [15]:
import networkx as nx

class Network:
    def __init__(self):
        self.edges = []
    def readFile(self,filename):
        with open(filename,'r') as f:
            for l in f:
                a,b = l.split("\t")
                e = (int(a),int(b))
                self.addEdge(e)
    def addEdge(self,e):
        self.edges.append(e)
    def __contains__(self,e):
        return e in self.edges
    def toNetworkX(self):
        n = nx.Graph()
        n.add_edges_from(self.edges)
        return n

In [16]:
dolphins = Network()
dolphins.readFile("dolphins.tsv")

Then we can use Bokeh to plot the network:

In [17]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx
output_notebook()

dolphins_nx=dolphins.toNetworkX()

plot = figure(plot_width=400, plot_height=400,x_range=(-2,2), y_range=(-2,2))

graph = from_networkx(dolphins_nx, nx.spring_layout, scale=2, center=(0,0))
graph.node_renderer.glyph.update(size=10, fill_color="orange")
plot.renderers.append(graph)
show(plot)

Loading BokehJS ...